In [1]:

# Default locations if parameter not passed
benchmark_data_path = "/scratch/rodrigo.freitas/remy-schedulers/tb-compare.pkl"


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])


rprint(df.columns)
rprint(df.head())
rprint(df.tail())

Index(['name', 'iter', 'full_output', 'tb_path', 'image', 'hosts',
       'ompc_tb_iter_num', 'tb-time', 'type', 'node_count', 'time'],
      dtype='object')

name   iter                                        full_output  \
0  tb-compare   1000  [1749581463.931383] [sorgan-cpu1:236584:0]    ...   
1  tb-compare   1000  [1749581466.834971] [sorgan-cpu1:236845:0]    ...   
2  tb-compare  10000  [1749581467.543674] [sorgan-cpu1:236965:0]    ...   
3  tb-compare  10000  [1749581468.240650] [sorgan-cpu1:237088:0]    ...   
4  tb-compare   1000  [1749581468.938838] [sorgan-cpu1:237207:0]    ...   

        tb_path               image  \
0  tb-main-loop  scheduler-main.sif   
1  tb-main-loop  scheduler-main.sif   
2  tb-main-loop  scheduler-main.sif   
3  tb-main-loop  scheduler-main.sif   
4  tb-main-loop  scheduler-main.sif   

                                             hosts  ompc_tb_iter_num  \
0  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4                10   
1  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4                10   
2  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4                10   
3  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4                10   
4  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4                10   

    tb-time type  node_count      time  
0  0.236044  fft           1  0.706640  
1  0.242522  fft           1  0.697195  
2  0.234584  fft           1  0.695917  
3  0.236863  fft           1  0.697394  
4  0.645873  fft           2  1.353310

name   iter                                        full_output  \
67  tb-compare  10000  [1749582011.895373] [sorgan-cpu1:246162:0]    ...   
68  tb-compare   1000  [1749582020.932916] [sorgan-cpu1:246409:0]    ...   
69  tb-compare   1000  [1749582032.060563] [sorgan-cpu1:246531:0]    ...   
70  tb-compare  10000  [1749582043.187919] [sorgan-cpu2:222352:0]    ...   
71  tb-compare  10000  [1749582077.588050] [sorgan-cpu2:222697:0]    ...   

           tb_path           image  \
67  tb-tdg-no-loop  improv-144.sif   
68  tb-tdg-no-loop  improv-144.sif   
69  tb-tdg-no-loop  improv-144.sif   
70  tb-tdg-no-loop  improv-144.sif   
71  tb-tdg-no-loop  improv-144.sif   

                                              hosts  ompc_tb_iter_num  \
67  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4               100   
68  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4               100   
69  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4               100   
70  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4               100   
71  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4               100   

      tb-time type  node_count       time  
67   3.958633  fft           2   4.431319  
68  10.642850  fft           4  11.121412  
69  10.646480  fft           4  11.124958  
70  10.711940  fft           4  11.185593  
71  10.710770  fft           4  11.191530

In [3]:
sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

In [4]:
def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [5]:
bench_names = df["name"].unique()
run_metadata = benchmark_data["config"].applications

for bench in bench_names:
    bench_metadata = run_metadata[bench]
    n_runs = benchmark_data["config"].metadata.runs
    print(f"Benchmark: {bench} - Runs: {n_runs}")
    # print(bench_metadata)
    df_bench = df[df["name"] == bench]
    for idx, row in df_bench.iterrows():
        print(f"--- Row {idx} ---")
        for col, val in row.items():
            print(f"{col}: {val}")   # printing val directly never truncates
        print()



Benchmark: tb-compare - Runs: 2
--- Row 0 ---
name: tb-compare
iter: 1000
full_output: [1749581463.931383] [sorgan-cpu1:236584:0]        ib_verbs.h:87   UCX  ERROR ibv_exp_query_device(mlx5_0) returned 95: Operation not supported
Running Task Benchmark
  Configuration:
    Task Graph 1:
      Time Steps: 32
      Max Width: 32
      Dependence Type: fft
      Radix: 3
      Period: 0
      Fraction Connected: 0.250000
      Kernel:
        Type: compute_bound
        Iterations: 1000
        Samples: 16
        Imbalance: 0.000000
      Output Bytes: 16
      Scratch Bytes: 0
Loop 0 : Elapsed time : 0.024403
Loop 1 : Elapsed time : 0.022082
Loop 2 : Elapsed time : 0.022360
Loop 3 : Elapsed time : 0.023046
Loop 4 : Elapsed time : 0.022887
Loop 5 : Elapsed time : 0.022918
Loop 6 : Elapsed time : 0.023012
Loop 7 : Elapsed time : 0.022698
Loop 8 : Elapsed time : 0.023097
Loop 9 : Elapsed time : 0.022702
Total Tasks 1024
Total Dependencies 2666
Total FLOPs 131137536
Total Bytes 0
Elapsed Ti